In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import data
from torchtext.datasets import PennTreebank

In [2]:
TEXT = data.Field(sequential=True, tokenize='spacy', lower=True)
train, valid, test = PennTreebank.splits(TEXT)
TEXT.build_vocab(train)
vocab_size = len(TEXT.vocab)

context_size = 2

train_iter, valid_iter, test_iter = data.BPTTIterator.splits((train, valid, test),
                                                            batch_size=512,
                                                            bptt_len=context_size * 2 + 1, # context on both sides, plus the center word
                                                            repeat=False)

downloading ptb.train.txt
downloading ptb.valid.txt
downloading ptb.test.txt


ptb.train.txt: 5.10MB [00:05, 963kB/s]
ptb.valid.txt: 400kB [00:00, 3.05MB/s]
ptb.test.txt: 450kB [00:01, 428kB/s]


In [3]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)
    
    def forward(self, inputs):
        out = torch.sum(self.embeddings(inputs), dim=0)
        out = self.linear(out)
        out = F.log_softmax(out, dim=0)
        return out

In [4]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)
    
    def forward(self, inputs):
        out = self.embeddings(inputs)
        out = self.linear(out)
        out = F.log_softmax(out, dim=0)
        return out

In [0]:
loss_function = nn.NLLLoss()
model = CBOW(vocab_size, 100)
optimizer = optim.SGD(model.parameters(), lr=1e-3)

for epoch in range(1):
    
    for i, batch in enumerate(train_iter):
        optimizer.zero_grad()
        
        text, target = batch.text, batch.text[context_size]
        output = model(text)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()